* rewriting the Zero-Pi Hamiltonian under ridge transformation
* Ridge transformation reduce the 4 dof to 2 dof
* more general approach to explicit transformation should be marked in the library
* this example serves as apt prototype
* comparison against  scQubits FullZeroPi

In [1]:
from DiSuQ.Torch import models
import scqubits
from torch.linalg import inv
from torch import tensor,stack,lobpcg,float,diag,sqrt
from numpy import arange,linspace,meshgrid,array,log,argsort,pi,roll,var,std,pi,exp
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracyTarget,Adam
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
im = 1.0j
set_num_threads(32)

In [2]:
L0 = indE(1e-6); L_ = indE(5e-2); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(1e-16); C_ = capE(1e-10) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(1e-16); CJ_ = capE(1e-10) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 50. ; J_ = 0. ; print('Junction Bound(GHz):',J_,J0+J_)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 3.269230258996635e-06 0.16346478218009075
Capacitance Bound(GHz): 0.00019370229307707598 193.70248677936902
Shunt Bound(GHz): 0.00019370229307707598 193.70248677936902
Junction Bound(GHz): 0.0 50.0


In [3]:
flux_grid = scqubits.Grid1d(-6*pi, 6*pi, 501)

In [4]:
Ep = 50
El = Ep/10000
Ej = Ep/7.9
EcS = Ep/2200
EcJ = Ep*Ep/Ej/8
Ec = 1/(1/EcS-1/EcJ)

In [5]:
indE(El),Ej,capE(EcJ),capE(Ec)

(3.269230258996635e-05,
 6.329113924050633,
 3.9230844167509056e-16,
 8.518977810974592e-13)

In [7]:
1/EcS,1/EcJ,1/Ec

(44.0, 0.020253164556962026, 43.97974683544304)

In [33]:
El*4*pi*pi*2

0.39478417604357435

In [9]:
basis = {'Chi':64,'Theta':32,'Phi':128}
rep = 'R'
circuit = models.zeroPi(basis,Ej=Ej,Ec=Ec,El=El,EcJ=EcJ,sparse=True,symmetry=True,_L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0),
                        ridge=True)

In [10]:
circuit.R

tensor([[ 0.5000, -0.5000, -0.5000],
        [-0.5000,  0.5000, -0.5000],
        [-0.5000, -0.5000,  0.5000]])

In [11]:
Ln_,Cn_ = circuit.Ln_,circuit.Cn_
R = circuit.R
L_ = inv(R.T) @ Ln_ @ inv(R); L_ = diag(diag(L_))
C_ = R @ Cn_ @ R.T; C_ = diag(diag(C_))

In [6]:
El,Ej,EcJ,Ec

(0.005, 6.329113924050633, 49.375, 0.022737738890168085)

In [12]:
Cn_

tensor([[4.9403e+01, 2.2730e-02, 4.9380e+01],
        [2.2730e-02, 4.5465e-02, 2.2740e-02],
        [4.9380e+01, 2.2740e-02, 4.9403e+01]], grad_fn=<LinalgInvExBackward0>)

In [13]:
L_,C_

(tensor([[0.3948, 0.0000, 0.0000],
         [0.0000, 0.3948, 0.0000],
         [0.0000, 0.0000, 0.0000]], grad_fn=<DiagEmbedBackward0>),
 tensor([[2.2739e-02, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 4.9380e+01, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 2.2728e-02]], grad_fn=<DiagEmbedBackward0>))

In [32]:
C_[:1,:1]

tensor([[0.0227]], grad_fn=<SliceBackward0>)

In [14]:
1/(1/C_[0,0]+1/C_[1,1])

tensor(0.0227, grad_fn=<MulBackward0>)

### specialized 0-pi Hamiltonian modules

In [15]:
self = circuit

In [16]:
flux0 = pi*6

#### H_LC

In [17]:
#def ridgeHamiltonian(self):
basis = self.basis
Ln_,Cn_ = self.Ln_,self.Cn_
R = self.R
L_ = inv(R.T) @ Ln_ @ inv(R); L_ = diag(diag(L_))
C_ = R @ Cn_ @ R.T; C_ = diag(diag(C_))
# Chi mode impedance
z = sqrt(C_[1,1]/L_[1,1]) * 2 # Cooper pair factor

F = [self.backend.basisFf(2*basis['Phi']+1,flux0)]
F += [self.backend.basisFo(basis['Chi'],z)]
F += [self.backend.basisFq(basis['Theta'])]

Q = [self.backend.basisFiniteII(2*basis['Phi']+1,(-flux0,flux0))]
Q += [self.backend.basisQo(basis['Chi'],z)]
Q += [self.backend.basisQq(basis['Theta'])]

/Users/chishti/JupyterHub/DiSuQ/Torch/sparse.py:179: UserWarning:

Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)



In [18]:
# explicit basis change
H_L = self.backend.modeMatrixProduct(F,L_,F,(0,0))/2

In [19]:
H_C = self.backend.modeMatrixProduct(Q,C_[1:,1:],Q,(1,1))/2

In [20]:
H_LC = H_L + H_C - C_[0,0]/2*self.backend.basisProduct(Q,[0])
#return H

In [21]:
H_LC

tensor(indices=tensor([[      0,       0,       1,  ..., 1069118, 1069119,
                        1069119],
                       [      0,     130,       1,  ..., 1069118, 1068989,
                        1069119]]),
       values=tensor([118.9863+0.j,   0.7453+0.j, 116.1226+0.j,  ...,
                        1.4272+0.j,  32.9384+0.j, 173.4462+0.j]),
       size=(1069120, 1069120), nnz=10553916, layout=torch.sparse_coo,
       grad_fn=<SubBackward0>)

In [22]:
# spectrum = lobpcg(H_LC.to(float),k=3,largest=False)[0]
# spectrum

##### H_Junction

In [23]:
def phase(phi):
    # phi = flux/flux_quanta
    return exp(im*2*pi*phi)

In [24]:
flux = .5

In [30]:
self.josephsonComponents()[0]

[(0, 1, 'Jx'), (2, 3, 'Jy')]

In [25]:
def ridgeJosephson(self):
    def Hamiltonian(flux):
        Ej = self.josephsonComponents()[1][0]
        D_Phi = self.backend.displacementFlux(2*basis['Phi']+1,flux0,1)*phase(flux/2) 
        D_Phi += self.backend.displacementFlux(2*basis['Phi']+1,flux0,-1)*phase(-flux/2) 
        D_Chi = self.backend.displacementOscillator(basis['Chi'],tensor(1.),1)
        D_Chi += self.backend.displacementOscillator(basis['Chi'],tensor(1.),-1)
        D_Theta = self.backend.chargeDisplacePlus(basis['Theta']) 
        D_Theta += self.backend.chargeDisplaceMinus(basis['Theta'])
        D = [D_Phi/2,D_Chi/2,D_Theta/2]
        H_J = -2*Ej*self.backend.basisProduct(D,[0,2])
        return H_J
    return Hamiltonian

In [26]:
circuit.ridgeJosephson = ridgeJosephson(circuit)

In [43]:
H_J = circuit.ridgeJosephson

In [44]:
H_J(flux)

tensor(indices=tensor([[      0,       1,       1,  ..., 1069118, 1069118,
                        1069119],
                       [      1,       0,       2,  ..., 1069117, 1069119,
                        1069118]]),
       values=tensor([ 0.4053+0.j,  0.4053+0.j,  0.4053+0.j,  ..., -0.4053+0.j,
                      -0.4053+0.j, -0.4053+0.j]),
       size=(1069120, 1069120), nnz=2105344, layout=torch.sparse_coo)

In [29]:
D = [D_Phi/2,D_Chi/2,D_Theta/2]

In [40]:
H_J = self.backend.basisProduct(D,[0,2])

In [41]:
H_J

tensor(indices=tensor([[      0,       1,       1,  ..., 1069118, 1069118,
                        1069119],
                       [      1,       0,       2,  ..., 1069117, 1069119,
                        1069118]]),
       values=tensor([ 0.4053+0.j,  0.4053+0.j,  0.4053+0.j,  ..., -0.4053+0.j,
                      -0.4053+0.j, -0.4053+0.j]),
       size=(1069120, 1069120), nnz=2105344, layout=torch.sparse_coo)

### Displacement Combination

In [28]:
circuit.josephsonComponents()

([(0, 1, 'Jx'), (2, 3, 'Jy')],
 [tensor(6.3291, grad_fn=<AddBackward0>),
  tensor(6.3291, grad_fn=<AddBackward0>)])

In [19]:
circuit.linearCombination(-1)

tensor([-1., -1., -0.])

In [21]:
circuit.linearCombination(0)

tensor([ 0., -1., -1.])

In [22]:
circuit.linearCombination(1)

tensor([-1., -0., -1.])

In [23]:
circuit.linearCombination(2)

tensor([-1., -1., -0.])

In [25]:
circuit.josephsonComponents()

([(0, 1, 'Jx'), (2, 3, 'Jy')],
 [tensor(6.3291, grad_fn=<AddBackward0>),
  tensor(6.3291, grad_fn=<AddBackward0>)])

In [27]:
circuit.nodes_

{1: 0, 2: 1, 3: 2, 0: -1}

In [ ]:
i,o,j = combination

In [ ]:
def displacementCombination(combination):
    self = circuit
    basis = self.basis
    R = self.R
    Ln_,Cn_ = self.Ln_,self.Cn_
    L_ = inv(R.T) @ Ln_ @ inv(R); L_ = diag(diag(L_))
    C_ = R @ Cn_ @ R.T; C_ = diag(diag(C_))
    # Chi mode impedance
    z = sqrt(C_[1,1]/L_[1,1]) * 2 # Cooper pair factor
    i,o,j = combination
    # re-calculation with parameter iteration
    D_plus = [self.backend.displacementFlux(basis['Phi'],i)] 
    D_minus = [self.backend.displacementFlux(basis['Phi'],-i)] 
    D_plus += [self.backend.displacementOscillator(basis['Chi'],z,o)]
    D_minus += [self.backend.displacementOscillator(basis['Chi'],z,-o)]
    D_plus += [self.backend.displacementCharge(basis['Theta'],j)]  
    D_minus += [self.backend.displacementCharge(basis['Theta'],-j)] 

    assert len(combination)==len(D_plus)
    assert len(combination)==len(D_minus)
    return D_plus,D_minus

In [ ]:
def kermanBasisSize():
    basis = circuit.basis
    return basis['Chi']*(2*basis['Theta']+1)*(2*basis['Phi']+1)

In [ ]:
circuit.displacementCombination = displacementCombination
circuit.ridgeHamiltonian = ridgeHamiltonian
circuit.kermanBasisSize = kermanBasisSize

In [8]:
H_J = circuit.kermanHamiltonianJosephson
H_LC = circuit.ridgeHamiltonian(circuit)

TypeError: linspace(): argument 'steps' (position 3) must be int, not float

In [ ]:
H_J({'Lx':tensor(0)})